# Metal Fuel Cycle
This notebook contains a IDAES model of the iron powder metal fuel cycle.  This cycle is being considered for use as a long term energy storage approach.

In [15]:
# IDAES / Pyomo imports
from pyomo.environ import (ConcreteModel, units as u, value, Var, Constraint, Objective, minimize)
from idaes.core import FlowsheetBlock
from idaes.core.util.scaling import calculate_scaling_factors
from idaes.core.util.initialization import propagate_state
from idaes.core.base.phases import Phase, LiquidPhase, VaporPhase, SolidPhase, AqueousPhase
from idaes.core.base.components import Component, Solvent, Solute
from idaes.models.unit_models import StoichiometricReactor, Heater, Mixer, Separator, Product, Feed
from idaes.models.properties.modular_properties.base.generic_property import GenericParameterBlock
from idaes.models.properties.modular_properties.eos.ideal import Ideal
from idaes.models.properties.modular_properties.phase_equil import SmoothVLE
from idaes.models.properties.modular_properties.reactions.dh_rxn import constant_dh_rxn
from idaes.models.properties.modular_properties.state_definitions import FTPx

from idaes.core.util.model_statistics import degrees_of_freedom

from idaes.core.solvers import get_solver
solver = get_solver("ipopt")

In [23]:
# 1) Property package configs

# Gas (ideal) for H2/H2O/O2/N2
gas_config = {
    "components": {
        "H2": {"type": Component, "elemental_composition": {"H":2}},
        "H2O": {"type": Component, "elemental_composition": {"H":2, "O":1}},
        "O2": {"type": Component, "elemental_composition": {"O":2}},
        "N2": {"type": Component, "elemental_composition": {"N":2}},
    },
    "phases": {"Vap": {"type": VaporPhase, "equation_of_state": Ideal}},
    "state_definition": FTPx,
    "base_units": {"time": u.s, "length": u.m, "mass": u.kg, "amount": u.mol, "temperature": u.K},
    "pressure_ref": 101325*u.Pa,
    "temperature_ref": 298.15*u.K,
    #"phases_in_equilibrium": [],
    "parameter_data": {},
}

# Simple solid phase for Fe / oxides (constant Cp & density here; swap to NASA/Cantera later)
solid_config = {
    "components": {
        "Fe": {"type": Component, "elemental_composition": {"Fe":1}},
        "Fe2O3": {"type": Component, "elemental_composition": {"Fe":2, "O":3}},
        "Fe3O4": {"type": Component, "elemental_composition": {"Fe":3, "O":4}},
    },
    "phases": {"Sol": {"type": SolidPhase, "equation_of_state": Ideal}},  # EoS unused for solids
    "state_definition": FTPx,
    "base_units": {"time": u.s, "length": u.m, "mass": u.kg, "amount": u.mol, "temperature": u.K},
    "pressure_ref": 101325*u.Pa,
    "temperature_ref": 298.15*u.K,
    #"phases_in_equilibrium": [],
    "parameter_data": {
        # (optional) add Cp/ρ as needed; IDAES generic lets you extend with specific heat models
    },
}

In [27]:
# Build Model
m = ConcreteModel()
m.fs = FlowsheetBlock(dynamic=False)

m.fs.props_gas = GenericParameterBlock(**gas_config)
m.fs.props_sol = GenericParameterBlock(**solid_config)

print(type(m.fs.props_gas))

<class 'idaes.core.base.process_block._ScalarGenericParameterBlock'>


In [32]:
# Small value for numerical stability
eps = 1e-9

# --- Feeds ---
# Steam feed to electrolyzer (can be saturated steam train in a richer model)
m.fs.feed_steam = Feed(property_package=m.fs.props_gas)
m.fs.feed_steam.flow_mol.fix(10)          # mol/s (tune)
m.fs.feed_steam.temperature.fix(373.15)   # K
m.fs.feed_steam.pressure.fix(2e5)         # Pa
m.fs.feed_steam.mole_frac_comp[0, "H2"].fix(eps)
m.fs.feed_steam.mole_frac_comp[0, "H2O"].fix(1.0)
m.fs.feed_steam.mole_frac_comp[0, "O2"].fix(eps)
m.fs.feed_steam.mole_frac_comp[0, "N2"].fix(eps)

# Air/O2 feed for combustor; here use pure O2 from electrolyzer for a closed O2 loop MVP
m.fs.feed_o2 = Feed(property_package=m.fs.props_gas)
m.fs.feed_o2.flow_mol.fix(5)            # mol/s (tune)
m.fs.feed_o2.temperature.fix(298.15)
m.fs.feed_o2.pressure.fix(2e5)
m.fs.feed_o2.mole_frac_comp[0,"O2"].fix(1.0)
for j in ("H2","H2O","N2"):
    m.fs.feed_o2.mole_frac_comp[0,j].fix(0)

# Solid Fe2O3 feed to regenerator (as a stream of solids; temperature set)
m.fs.feed_ore = Feed(property_package=m.fs.props_sol)
m.fs.feed_ore.flow_mol.fix(1.5)           # mol/s Fe2O3
m.fs.feed_ore.temperature.fix(900)        # K (tune for kinetics/thermo)
m.fs.feed_ore.pressure.fix(2e5)
m.fs.feed_ore.mole_frac_comp[0,"Fe2O3"].fix(1.0)
m.fs.feed_ore.mole_frac_comp[0,"Fe"].fix(eps)
m.fs.feed_ore.mole_frac_comp[0,"Fe3O4"].fix(eps)

# --- Electrolyzer (as StoichiometricReactor with power / Faradaic constraint) ---
if m.fs.props_gas is None:
  print("Gas property package is None.")
m.fs.ely = StoichiometricReactor(property_package=m.fs.props_gas,
                                 has_heat_transfer=True, has_pressure_change=False)

# Reaction: H2O -> H2 + 0.5 O2  (we set stoich via conversion constraint below)
ely_rxn = {("H2O", -1.0), ("H2", 1.0), ("O2", 0.5)}
m.fs.ely.reaction_stoichiometry = {None: {("Vap", s): nu for (s, nu) in ely_rxn}}
m.fs.ely.conversion = Var(initialize=0.7, bounds=(0,1))
m.fs.ely.deltaG_split = Var(initialize=237e3, units=u.J/u.mol)  # ΔG H2O->H2+0.5O2 at ~25C
m.fs.ely.eta_elec = Var(initialize=0.75, bounds=(0.3, 1.0))
m.fs.ely.P_elec = Var(initialize=3e6, units=u.W)  # electrical power input

# Tie feed to electrolyzer inlet
from idaes.core import Arc
m.fs.s01 = Arc(source=m.fs.feed_steam.outlet, destination=m.fs.ely.inlet)

# Convert a fraction of H2O according to conversion variable
def _ely_material_bal(m, j):
    # outlet = inlet + nu * extent; here extent = conv * inlet_H2O_flow
    nin = m.fs.ely.inlet.flow_mol[0] * m.fs.ely.inlet.mole_frac_comp[0, j]
    nH2O_in = m.fs.ely.inlet.flow_mol[0] * m.fs.ely.inlet.mole_frac_comp[0, "H2O"]
    nu = dict(ely_rxn).get(j, 0.0)
    nout = m.fs.ely.outlet.flow_mol[0] * m.fs.ely.outlet.mole_frac_comp[0, j]
    return nout == nin + nu * m.fs.ely.conversion * nH2O_in
m.fs.ely.material_bal = Constraint(m.fs.props_gas.component_list, rule=_ely_material_bal)

# Power-to-H2 linkage:  P * eta / ΔG = molar extent = conv * n_H2O_in
m.fs.ely.power_link = Constraint(expr =
    m.fs.ely.P_elec * m.fs.ely.eta_elec / m.fs.ely.deltaG_split ==
    m.fs.ely.conversion * m.fs.ely.inlet.flow_mol[0] * m.fs.ely.inlet.mole_frac_comp[0,"H2O"]
)

# Heat duty variable from energy balance (sign convention: positive = heat added to reactor)
# Here we’ll let solver compute heat (ely_endotherm). You can also fix temperature targets via a Heater.

# --- Regenerator: Fe2O3 + 3 H2 -> 2 Fe + 3 H2O (gas-solid reactor) ---
# Gas side to reducer
m.fs.mix_red = Mixer(property_package=m.fs.props_gas, momentum_mixing_type=None, inlet_list=["h2","steam_from_ely"])
m.fs.s02 = Arc(source=m.fs.ely.outlet, destination=m.fs.mix_red.steam_from_ely)
# Add a small H2 makeup feed (optional); or split from electrolyzer
m.fs.feed_h2 = Feed(property_package=m.fs.props_gas)
m.fs.feed_h2.flow_mol.fix(3.0)
m.fs.feed_h2.temperature.fix(700)
m.fs.feed_h2.pressure.fix(2e5)
m.fs.feed_h2.mole_frac_comp[0,"H2"].fix(1.0)
for j in ("H2O","O2","N2"):
    m.fs.feed_h2.mole_frac_comp[0,j].fix(eps)
m.fs.s03 = Arc(source=m.fs.feed_h2.outlet, destination=m.fs.mix_red.h2)

# Solid+gas “reactor”: we model gas and solid separately with two SReactor blocks sharing a conversion.
m.fs.reductor_g = StoichiometricReactor(property_package=m.fs.props_gas,
                                        has_heat_transfer=True, has_pressure_change=False)
m.fs.reductor_s = StoichiometricReactor(property_package=m.fs.props_sol,
                                        has_heat_transfer=True, has_pressure_change=False)

# Gas stoich: -3 H2 + 3 H2O
red_rxn_g = {("H2",-3.0), ("H2O",3.0)}
m.fs.reductor_g.reaction_stoichiometry = {None: {("Vap", s): nu for (s,nu) in red_rxn_g}}
# Solid stoich: -1 Fe2O3 + 2 Fe
red_rxn_s = {("Fe2O3",-1.0), ("Fe",2.0)}
m.fs.reductor_s.reaction_stoichiometry = {None: {("Sol", s): nu for (s,nu) in red_rxn_s}}

# One shared conversion variable on Fe2O3
m.fs.x_red = Var(initialize=0.9, bounds=(0,1))

# Connect streams
m.fs.s04 = Arc(source=m.fs.mix_red.outlet, destination=m.fs.reductor_g.inlet)
m.fs.s05 = Arc(source=m.fs.feed_ore.outlet,   destination=m.fs.reductor_s.inlet)

# Material balances via conversion
def _red_bal_g(m, j):
    nin = m.fs.reductor_g.inlet.flow_mol[0]*m.fs.reductor_g.inlet.mole_frac_comp[0,j]
    nFe2O3 = m.fs.reductor_s.inlet.flow_mol[0]*m.fs.reductor_s.inlet.mole_frac_comp[0,"Fe2O3"]
    nu = dict(red_rxn_g).get(j, 0.0)
    nout = m.fs.reductor_g.outlet.flow_mol[0]*m.fs.reductor_g.outlet.mole_frac_comp[0,j]
    return nout == nin + nu * m.fs.x_red * nFe2O3
m.fs.reductor_g.mb = Constraint(m.fs.props_gas.component_list, rule=_red_bal_g)

def _red_bal_s(m, j):
    nin = m.fs.reductor_s.inlet.flow_mol[0]*m.fs.reductor_s.inlet.mole_frac_comp[0,j]
    nFe2O3 = m.fs.reductor_s.inlet.flow_mol[0]*m.fs.reductor_s.inlet.mole_frac_comp[0,"Fe2O3"]
    nu = dict(red_rxn_s).get(j, 0.0)
    nout = m.fs.reductor_s.outlet.flow_mol[0]*m.fs.reductor_s.outlet.mole_frac_comp[0,j]
    return nout == nin + nu * m.fs.x_red * nFe2O3
m.fs.reductor_s.mb = Constraint(m.fs.props_sol.component_list, rule=_red_bal_s)

# Optional: condenser to remove steam & recycle H2, omitted here for brevity

# --- Combustor: 3 Fe + 2 O2 -> Fe3O4 + heat ---
m.fs.combust_g = StoichiometricReactor(property_package=m.fs.props_gas,
                                       has_heat_transfer=True, has_pressure_change=False)
m.fs.combust_s = StoichiometricReactor(property_package=m.fs.props_sol,
                                       has_heat_transfer=True, has_pressure_change=False)

rxn_c_g = {("O2",-2.0)}               # gas side O2 consumption only
rxn_c_s = {("Fe",-3.0), ("Fe3O4",1.0)}
m.fs.combust_g.reaction_stoichiometry = {None: {("Vap", s): nu for (s,nu) in rxn_c_g}}
m.fs.combust_s.reaction_stoichiometry = {None: {("Sol", s): nu for (s,nu) in rxn_c_s}}

# Connect O2 and Fe streams
m.fs.s06 = Arc(source=m.fs.reductor_s.outlet, destination=m.fs.combust_s.inlet)  # Fe product goes to combustor
m.fs.s07 = Arc(source=m.fs.feed_o2.outlet,    destination=m.fs.combust_g.inlet)

# --- Power block (black box): W_out = eta_pb * (Q_combust_total) ---
m.fs.eta_pb = Var(initialize=0.35, bounds=(0.1,0.7))
m.fs.W_out  = Var(initialize=1e6, units=u.W)

# Combustor duties (negative if exothermic to surroundings).
# Define a simple constraint summing both gas & solid reactor heat duties:
m.fs.heat_to_power = Constraint(expr = m.fs.W_out == - m.fs.eta_pb * (
    m.fs.combust_g.heat_duty[0] + m.fs.combust_s.heat_duty[0])
)

# --- Objective: minimize electrolyzer power for a target W_out (or solve feasibility)
target_power = 1.0e6  # 1 MW net electric out (tune)
m.fs.target = Constraint(expr = m.fs.W_out == target_power * u.W)
m.obj = Objective(expr = m.fs.ely.P_elec, sense=minimize)



'idaes.core.base.process_block._ScalarFeed'>) on block fs with a new Component
(type=<class 'idaes.core.base.process_block._ScalarFeed'>). This is usually
indicative of a modelling error. To avoid this warning, use
block.del_component() and block.add_component().
'idaes.core.base.process_block._ScalarFeed'>) on block fs with a new Component
(type=<class 'idaes.core.base.process_block._ScalarFeed'>). This is usually
indicative of a modelling error. To avoid this warning, use
block.del_component() and block.add_component().
WARNING (W1002): Setting Var 'fs.feed_o2.properties[0.0].mole_frac_comp[H2]'
to a numeric value `0` outside the bounds (1e-20, 1.001).
    See also https://pyomo.readthedocs.io/en/stable/errors.html#w1002
WARNING (W1002): Setting Var 'fs.feed_o2.properties[0.0].mole_frac_comp[H2O]'
to a numeric value `0` outside the bounds (1e-20, 1.001).
    See also https://pyomo.readthedocs.io/en/stable/errors.html#w1002
WARNING (W1002): Setting Var 'fs.feed_o2.properties[0.0].mole

AttributeError: 'NoneType' object has no attribute 'build_reaction_block'

In [ ]:
# Initialization / Solve
assert degrees_of_freedom(m) == 0

# Initialize in dataflow order
for blk in (m.fs.feed_steam, m.fs.feed_o2, m.fs.feed_ore, m.fs.feed_h2):
    blk.initialize()

propagate_state(m.fs.s01)
m.fs.ely.initialize()

propagate_state(m.fs.s03); propagate_state(m.fs.s02)
m.fs.mix_red.initialize()

propagate_state(m.fs.s04)
m.fs.reductor_g.initialize()

propagate_state(m.fs.s05)
m.fs.reductor_s.initialize()

propagate_state(m.fs.s06); propagate_state(m.fs.s07)
m.fs.combust_g.initialize()
m.fs.combust_s.initialize()

results = solver.solve(m, tee=False)
print(results.solver.termination_condition)
print("Electrolyzer power [MW]:", value(m.fs.ely.P_elec)/1e6)
print("Net electric out [MW]:", value(m.fs.W_out)/1e6)
print("Combustor duty total [MW]:", -value(m.fs.combust_g.heat_duty[0]+m.fs.combust_s.heat_duty[0])/1e6)